In [133]:
#libs
import pandas as pd 
import os 
import numpy as np
from datetime import date

In [95]:
#importando dados de contratos negociados e preço das cotações
df_ct = pd.read_pickle('cotacao_commodities.pkl')
df_tm = pd.read_pickle('negociacao_derivativos_commodities.pkl')

In [4]:
#dataframe cotações por mercadoria e bolsa negociadas
df_ct.head(2)

,data_cotacao,cod_ric,cod_ticker,cotacao_tipo,fator_conversao,bolsa_negociacao,tipo_mercadoria,mercadoria,moeda,unidade_mercadoria,valor_cotacao
0,2019-08-23,BACAINDXF,BACAINDX,FECHAMENTO,1.0,BMF,GORDO - ESALQ SÃO PAULO,BOI,REAL,ARROBA LIQUIDA,156.10
1,2019-08-23,ICFU19,AXU9,AJUSTE,1.0,BMF,ARABICO EM GRAOS- TIPO 4/5,CAFE,DOLAR DOS EUA,SACA(60KG),111.35


In [5]:
#tipo das variáveis 
df_ct.dtypes

data_cotacao          datetime64[ns]
cod_ric                       object
cod_ticker                    object
cotacao_tipo                  object
fator_conversao              float64
bolsa_negociacao              object
tipo_mercadoria               object
mercadoria                    object
moeda                         object
unidade_mercadoria            object
valor_cotacao                float64
dtype: object

In [6]:
#dataframe contratos de derivativos de commodities
df_tm.head(2)

,data_referencia,data_inicio,data_vencimento,numero_contrato,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,vendedor,comprador
0,2019-07-18,2019-07-17,2019-08-30,77902613,BORRACHA,TSR20,SGX,TONELADA METRICA,(220) USD,2,Cliente 1,Mercado
1,2019-07-18,2019-07-18,2020-01-14,77902616,MILHO,EM GRAOS A GRANEL,BMF,SACA(60KG),(790) BRL,11250,Mercado,Cliente 1


In [7]:
#tipo das variaveis
df_tm.dtypes

data_referencia         datetime64[ns]
data_inicio             datetime64[ns]
data_vencimento         datetime64[ns]
numero_contrato                 object
mercadoria                      object
tipo                            object
bolsa                           object
unidade_negociacao              object
moeda                           object
quantidade_negociada             int64
vendedor                        object
comprador                       object
dtype: object

In [96]:
#Tratando features de vendedor/comprador e criando coluna de cross das duas features
df_tm['vendedor'][df_tm['vendedor'] == 'Cliente 1'] = 'Cliente'
df_tm['comprador'][df_tm['comprador'] == 'Cliente 1'] = 'Cliente'
df_tm['comprador_vendedor'] = df_tm[['comprador', 'vendedor']].apply(lambda x: ' x '.join(x.dropna()), axis=1)

<ipython-input-96-2614472d6afc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tm['vendedor'][df_tm['vendedor'] == 'Cliente 1'] = 'Cliente'
<ipython-input-96-2614472d6afc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tm['comprador'][df_tm['comprador'] == 'Cliente 1'] = 'Cliente'


In [11]:
df_tm.head(2)

,data_referencia,data_inicio,data_vencimento,numero_contrato,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,vendedor,comprador,comprador_vendedor
0,2019-07-18,2019-07-17,2019-08-30,77902613,BORRACHA,TSR20,SGX,TONELADA METRICA,(220) USD,2,Cliente,Mercado,Mercado x Cliente
1,2019-07-18,2019-07-18,2020-01-14,77902616,MILHO,EM GRAOS A GRANEL,BMF,SACA(60KG),(790) BRL,11250,Mercado,Cliente,Cliente x Mercado


In [13]:
df_ct.head(2)

,data_cotacao,cod_ric,cod_ticker,cotacao_tipo,fator_conversao,bolsa_negociacao,tipo_mercadoria,mercadoria,moeda,unidade_mercadoria,valor_cotacao
0,2019-08-23,BACAINDXF,BACAINDX,FECHAMENTO,1.0,BMF,GORDO - ESALQ SÃO PAULO,BOI,REAL,ARROBA LIQUIDA,156.10
1,2019-08-23,ICFU19,AXU9,AJUSTE,1.0,BMF,ARABICO EM GRAOS- TIPO 4/5,CAFE,DOLAR DOS EUA,SACA(60KG),111.35


In [97]:
#tratamento das chaves de união das bases de cmm e ctc
lista_tm = ['mercadoria','tipo','bolsa','unidade_negociacao','moeda']
for l in lista_tm:
    df_tm[l] = df_tm[l].str.strip()

lista_ct = ['bolsa_negociacao','tipo_mercadoria','mercadoria','moeda','unidade_mercadoria']    
for l in lista_ct:
    df_ct[l] = df_ct[l].str.strip()


In [98]:
#merge dos datasets
df_merge = df_tm.merge(df_ct, how='inner', 
                      left_on=(['data_referencia','mercadoria','tipo','bolsa','unidade_negociacao']),
                      right_on=(['data_cotacao','mercadoria','tipo_mercadoria','bolsa_negociacao','unidade_mercadoria']))

In [18]:
#imprimindo as colunas do dataframe resultado do merge 
df_merge.columns

Index(['data_referencia', 'data_inicio', 'data_vencimento', 'numero_contrato',
       'mercadoria', 'tipo', 'bolsa', 'unidade_negociacao', 'moeda_x',
       'quantidade_negociada', 'vendedor', 'comprador', 'comprador_vendedor',
       'data_cotacao', 'cod_ric', 'cod_ticker', 'cotacao_tipo',
       'fator_conversao', 'bolsa_negociacao', 'tipo_mercadoria', 'moeda_y',
       'unidade_mercadoria', 'valor_cotacao'],
      dtype='object')

In [99]:
#deletando as colunas repetidas
df = df_merge.drop(columns=(['data_inicio','numero_contrato','data_vencimento','vendedor','comprador','data_cotacao',
                               'cotacao_tipo','bolsa_negociacao','tipo_mercadoria','moeda_y','unidade_mercadoria']))
#renomeando a coluna moeda
df.rename(columns={'moeda_x':'moeda'}, inplace=True)

In [30]:
df.head(2)

,data_referencia,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,fator_conversao,valor_cotacao
0,2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,1.0,1.9152
1,2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,1.0,1.8636


- Para obter o valor real dos contratos de derivativos na moeda brasileira (BRL) e fazer estudos da variação de câmbio, torna-se necessário enriquecer a nossa base de dados com cotação diária do dolar. 
- Além disso, para correção real do valor negociado com contratos passados, será adicionada a variável de ipca mensal, índice que mensura a inflação nacional. 

- Os dados são obtidos a partir de uma biblioteca denominada bcb.
    documentação: https://pypi.org/project/python-bcb/

In [1]:
#importar dados de usd
#Conversor de Moedas: O Conversor de Moedas realiza o Web Scrapping do sistema de mesmo nome do Banco Central. Permite que seja importado as cotações diárias de diversas moedas diante do real. Para saber as moedas que são possíveis de importar, e consequentemente seus respectivos símbolos, utiliza-se a função currency.get_currency_list().
from bcb import currency
from bcb import sgs

lista_moeda = currency.get_currency_list()

In [2]:
#criando um dataframe com dados diários do dolar
usd = currency.get(['USD'], start='2019-01-01', end='2022-07-14')
usd.head(2)

,USD
Date,
2019-01-02,3.8595
2019-01-03,3.7683


In [6]:
usd.dtypes

USD    float64
dtype: object

In [7]:
#criando dataframe com dados mensais de ipca 
ipca = sgs.get({'IPCA':433}, start='2019-01-01', end='2022-07-01')
#adicionando uma coluna do acumulado do ipca
ipca['ipca_acumulado'] = ipca['IPCA'].cumsum()
ipca.head(2)

,IPCA,ipca_acumulado
Date,,
2019-01-01,0.32,0.32
2019-02-01,0.43,0.75


In [8]:
ipca.dtypes

IPCA              float64
ipca_acumulado    float64
dtype: object

In [105]:
#adicionando data_referencia como index do dataframe principal para adicionar o dolar por dia 
df.set_index('data_referencia', inplace=True)

In [106]:
df.head(2)

,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,fator_conversao,valor_cotacao
data_referencia,,,,,,,,,,,
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,1.0,1.9152
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,1.0,1.8636


In [107]:
#adicionando coluna de usd no dataframe principal
df['usd'] = usd['USD']

In [108]:
#criando coluna de volume (notional) em usd e brl
df['notional_usd'] = df['quantidade_negociada']*df['valor_cotacao']
df['notional_brl'] = np.where(df['moeda']=='(220) USD', df['notional_usd']*df['usd'], df['notional_usd'])

In [109]:
df.head(2)

,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,fator_conversao,valor_cotacao,usd,notional_usd,notional_brl
data_referencia,,,,,,,,,,,,,,
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,1.0,1.9152,3.7596,6595948.8,2.479813e+07
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,1.0,1.8636,3.7596,6418238.4,2.413001e+07


In [ ]:
#resetando o index para criar coluna anomes
ipca.reset_index(inplace=True)
df.reset_index(inplace=True)

In [131]:
#criando coluna de ano e mes 
ipca['mes'] = ipca.Date.dt.month.astype(str)
ipca['ano'] = ipca.Date.dt.year.astype(str)
ipca['anomes'] = ipca[['ano', 'mes']].apply(lambda x: ''.join(x.dropna()), axis=1)
# dataframe principal
df['mes'] = df.data_referencia.dt.month.astype(str)
df['ano'] = df.data_referencia.dt.year.astype(str)
df['anomes'] = df[['ano', 'mes']].apply(lambda x: ''.join(x.dropna()), axis=1)

In [140]:
#setando index para juntar os dois dataframes
df.set_index('anomes', inplace=True)
ipca.set_index('anomes', inplace=True)

In [147]:
#adicionando dados do ipca no dataframe principal
df['ipca'] = ipca['IPCA']
df['ipca_acumulado'] = ipca['ipca_acumulado']

In [150]:
df.head(2)

,data_referencia,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,fator_conversao,valor_cotacao,usd,notional_usd,notional_brl,mes,ano,ipca,ipca_acumulado
anomes,,,,,,,,,,,,,,,,,,,
20197,2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,1.0,1.9152,3.7596,6595948.8,2.479813e+07,7,2019,0.19,2.4
20197,2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,1.0,1.8636,3.7596,6418238.4,2.413001e+07,7,2019,0.19,2.4


In [158]:
df.reset_index(inplace=True)
df.set_index('data_referencia', inplace=True)

In [162]:
#salvando dataframe principal
df.to_pickle('mastertable.pkl')

In [155]:
df['notionalusd_norm'] = (df['notional_usd'] - df['notional_usd'].mean())/df['notional_usd'].std()

In [159]:
df.head(2)

,anomes,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,...,usd,notional_usd,notional_brl,mes,ano,ipca,ipca_acumulado,notional_ipca,notionalusd_ipca,notionalusd_norm
data_referencia,,,,,,,,,,,,,,,,,,,,,
2019-07-24,20197,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,...,3.7596,6595948.8,2.479813e+07,7,2019,0.19,2.4,522179.28,522179.28,0.333448
2019-07-24,20197,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,...,3.7596,6418238.4,2.413001e+07,7,2019,0.19,2.4,508110.54,508110.54,0.319319
